### Recommendations with MovieTweetings: Getting to Know The Data

Throughout this lesson, you will be working with the [MovieTweetings Data](https://github.com/sidooms/MovieTweetings/tree/master/recsyschallenge2014).  To get started, you can read more about this project and the dataset from the [publication here](http://crowdrec2013.noahlab.com.hk/papers/crowdrec2013_Dooms.pdf).

**Note:** There are solutions to each of the notebooks available by hitting the orange jupyter logo in the top left of this notebook.  Additionally, you can watch me work through the solutions on the screencasts that follow each workbook. 

To get started, read in the libraries and the two datasets you will be using throughout the lesson using the code below.

 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t

%matplotlib inline

# Read in the datasets
movies = pd.read_csv('https://raw.githubusercontent.com/sidooms/MovieTweetings/master/latest/movies.dat', delimiter='::', header=None, names=['movie_id', 'movie', 'genre'], dtype={'movie_id': object}, engine='python')
reviews = pd.read_csv('https://raw.githubusercontent.com/sidooms/MovieTweetings/master/latest/ratings.dat', delimiter='::', header=None, names=['user_id', 'movie_id', 'rating', 'timestamp'], dtype={'movie_id': object, 'user_id': object, 'timestamp': object}, engine='python')

#### 1. Take a Look At The Data 

Take a look at the data and use your findings to fill in the dictionary below with the correct responses to show your understanding of the data.

In [2]:
# The number of movies in the dataset
movies.movie_id.nunique()

32661

In [3]:
# The number of ratings in the dataset
reviews.drop_duplicates().shape

(754464, 4)

In [4]:
# The number of different genre
movies.genre.nunique()

1226

In [5]:
# The number of unique users in the dataset
reviews.user_id.nunique()

56066

In [6]:
# The number missing ratings in the reviews dataset
reviews.rating.isnull().sum()

0

In [7]:
# The average rating given across all ratings
reviews.rating.mean()

7.2995729418501085

In [8]:
# The minimum rating given across all ratings
reviews.rating.min()

0

In [9]:
# The maximum rating given across all ratings
reviews.rating.max()

10

In [10]:
# Use your findings to match each variable to the correct statement in the dictionary


dict_sol1 = {
'The number of movies in the dataset': 32661,
'The number of ratings in the dataset': 754464,
'The number of different genres': 1226,
'The number of unique users in the dataset': 56066,
'The number missing ratings in the reviews dataset': 0,
'The average rating given across all ratings': 7.29,
'The minimum rating given across all ratings': 0,
'The maximum rating given across all ratings': 10
}

# Originally, I had this to check your solution, but the 
# links are live and updating.  That didn't end up being
# a great idea


#### 2. Data Cleaning

Next, we need to pull some additional relevant information out of the existing columns. 

For each of the datasets, there are a couple of cleaning steps we need to take care of:

#### Movies
* Pull the date from the title and create new column
* Dummy the date column with 1's and 0's for each century of a movie (1800's, 1900's, and 2000's)
* Dummy column the genre with 1's and 0's

#### Reviews
* Create a date out of time stamp

You can check your results against the header of my solution by running the cell below with the **show_clean_dataframes** function.

In [11]:
# create date column
create_date = lambda x: x[-5:-1] if x[-1]==")" else np.nan
movies['date'] = movies['movie'].apply(create_date)
movies.head()

,movie_id,movie,genre,date
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894
1,0000010,La sortie des usines Lumière (1895),Documentary|Short,1895
2,0000012,The Arrival of a Train (1896),Documentary|Short,1896
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895
4,0000091,Le manoir du diable (1896),Short|Horror,1896


In [12]:
# create Dummy the date column with 1's and 0's for each century of a movie (1800's, 1900's, and 2000's)
def add_movie_year(val):
    if val[:2] == yr:
        return 1
    else:
        return 0
    
for yr in ['18', '19', '20']:
    movies[yr+"00's"] = movies['date'].apply(add_movie_year)

movies.head()

,movie_id,movie,genre,date,1800's,1900's,2000's
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0
1,0000010,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0
2,0000012,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0
4,0000091,Le manoir du diable (1896),Short|Horror,1896,1,0,0


In [13]:
# create Dummy column the genre with 1's and 0's

# get movie genres
genres = []

for val in movies['genre']:
    try:
        genres.extend(val.split("|"))
        
    except AttributeError:
        pass
    
genres = set(genres)
print(genres)

{'Short', 'Musical', 'Action', 'Reality-TV', 'Film-Noir', 'Comedy', 'History', 'Thriller', 'Romance', 'Adventure', 'Game-Show', 'Sport', 'Documentary', 'News', 'Mystery', 'Family', 'Biography', 'Horror', 'Adult', 'Sci-Fi', 'Music', 'Drama', 'Animation', 'Crime', 'War', 'Talk-Show', 'Western', 'Fantasy'}


In [14]:
def split_genres(val):
    try:
        if val.find(g) > -1:
            return 1
        else:
            return 0
    except AttributeError:
        return 0

In [15]:
%%time
for g in genres:
    movies[g] = movies['genre'].apply(split_genres)

CPU times: user 630 ms, sys: 220 µs, total: 630 ms
Wall time: 635 ms


In [16]:
movies.head()

,movie_id,movie,genre,date,1800's,1900's,2000's,Short,Musical,Action,...,Adult,Sci-Fi,Music,Drama,Animation,Crime,War,Talk-Show,Western,Fantasy
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0000010,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0000012,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0000091,Le manoir du diable (1896),Short|Horror,1896,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
# change timestamp to datetime
import datetime

change_timestamp = lambda x: datetime.datetime.fromtimestamp(int(x)).strftime("%Y-%m-%d %H:%M:%S")

reviews["date_time"] = reviews["timestamp"].apply(change_timestamp)

In [18]:
reviews.head()

,user_id,movie_id,rating,timestamp,date_time
0,1,0068646,10,1381620027,2013-10-12 23:20:27
1,1,0113277,10,1379466669,2013-09-18 01:11:09
2,2,0422720,8,1412178746,2014-10-01 15:52:26
3,2,0454876,8,1394818630,2014-03-14 17:37:10
4,2,0790636,7,1389963947,2014-01-17 13:05:47


In [19]:
reviews_new, movies_new = t.show_clean_dataframes()

   Unnamed: 0  user_id  movie_id  rating   timestamp                 date  \
0           0        1     68646      10  1381620027  2013-10-12 23:20:27   
1           1        1    113277      10  1379466669  2013-09-18 01:11:09   
2           2        2    422720       8  1412178746  2014-10-01 15:52:26   
3           3        2    454876       8  1394818630  2014-03-14 17:37:10   
4           4        2    790636       7  1389963947  2014-01-17 13:05:47   

   month_1  month_2  month_3  month_4    ...      month_9  month_10  month_11  \
0        0        0        0        0    ...            0         1         0   
1        0        0        0        0    ...            0         0         0   
2        0        0        0        0    ...            0         1         0   
3        0        0        0        0    ...            0         0         0   
4        0        0        0        0    ...            0         0         0   

   month_12  year_2013  year_2014  year_2015  year